## 用电量预测
根据某客户2006年电量数据（见附件），⽤机器学习模型来预测未来⼀小时的电量和未来6小时的⽤电量。数据文件中“time”列是时间戳（⽇期及相应的⼩时、分钟），“KWH”是每个时间戳对应的那个小时⾥的用电量。具体来讲，在每⼀个⼩小时t, 我们需要预测（t+1）⼩时的⽤电量q(t+1)。
使用**Mean Absolute Percentage Error**来展示所建模型的预测效果。

- 使用XGBoost模型来建⽴立预测模型。
- 使用Long Short Term Memory模型来建⽴立预测模型。
- 解决预测⼀个6个元素的向量[q(t+1), q(t+2), q(t+3), q(t+4), q(t+5), q(t+6)]的问题。


*对于XGBOOST选择了1，3小时前，1，3，7天前的电量，月，日，时作为特征，最后训练集MAPE为0.3%，测试集MAPE为3.1&
对于LSTM选择了1，3，7天前的电量，月，日，时等几个特征，时间滑窗为6个小时，预测未来6个小时的耗电量，最后训练集MAPE为8.5%，测试集MAPE为8.3%
虽然结果显示XGBOOST效果更好，但是对于这类时间序列问题，可以预见在超参数及特征选择优化后，LSTM的效果会达到XGBOOST同样的水平，甚至超过.*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from datetime import date, timedelta
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline

In [285]:
data = pd.read_csv('data.csv',parse_dates = ['time'],index_col = 'time')

In [287]:
data = data.drop('Unnamed: 0',axis = 1)
data_prep = data.copy()

In [288]:
def hour_shift(data_prep,df,n):
    if n> 0:
        data_prep[f'{n}_hour_before'] = df['KWH'].shift(n)
    else:
        data_prep[f'{-n}_hour_after'] = df['KWH'].shift(n)

In [289]:
def days_shift(data_prep,df,n):
    if n> 0:
        data_prep[f'{n}_days_before'] = data.loc[data.index - timedelta(days=n)]['KWH'].values
    else:
        data_prep[f'{-n}_days_after'] = data.loc[data.index + timedelta(days=-n)]['KWH'].values

In [290]:
hours_shift_list = [-6,-5,-4,-3,-2,-1,1,3]
days_shift_list = [1,3,7]
for hours in hours_shift_list:
    hour_shift(data_prep,data,hours)
for days in days_shift_list:
    days_shift(data_prep,data,days)
    

C:\Users\Administrator\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


In [291]:
def add_more_features(df):
    """
    Add 'month','day','hour' features
    """
    data_prep = df.copy()
    data_prep['month'] = df.index.month
    data_prep['day'] = df.index.day
    data_prep['hour'] = df.index.hour.astype(int)+1
    return data_prep

In [292]:
data_prep = add_more_features(data_prep)

In [293]:
data_prep = data_prep.dropna(axis = 0)

In [296]:
def mape(y_pred,y_label):
    """
    Define Mean Absolute Percentage Error
    """
    
    mape = np.sum(abs(y_pred - y_label)) / np.sum(y_label)
    return mape
    

In [379]:
# XGB
xgb_features = ['KWH','1_hour_before','3_hour_before','1_days_before','3_days_before','7_days_before','month','day','hour']
xgb_target = ['1_hour_after']

X_train,X_test,y_train,y_test = train_test_split(data_prep[xgb_features],
                                                data_prep[xgb_target],
                                                test_size = 0.2,
                                                random_state = 8896)

In [342]:
myxgb = XGBRegressor()

depth_range = np.arange(7,10,1)
num_estimators = np.arange(300,500,50)
grid_params = {
    'max_depth':depth_range,
    'n_estimators':num_estimators
}

gsearch = GridSearchCV(myxgb,
                       grid_params,
                       scoring = 'neg_mean_absolute_error',
                      verbose = 1,
                      n_jobs = -1,
                       cv = 5,
                      refit = True)


In [343]:
print('start training..')
gsearch.fit(X_train,y_train)
print('train finished!')

start training..
Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   27.9s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   46.5s finished


[19:03:49] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train finished!


In [344]:
model = gsearch.best_estimator_

In [345]:
y_predict_validation = model.predict(X_test)
y_predict_train = model.predict(X_train)


In [346]:
xgb_score_validation = mape(y_predict_validation,y_test.values.reshape(1,-1))
xgb_score_train = mape(y_predict_train,y_train.values.reshape(1,-1))
print(f'The Mean_Percentage_Absolute_Error of the XGB model on validation set is {round(xgb_score_validation,3) * 100}%')
print(f'The Mean_Percentage_Absolute_Error of the XGB model on train set is {round(xgb_score_train,3) * 100}%')


The Mean_Percentage_Absolute_Error of the XGB model on validation set is 3.1%
The Mean_Percentage_Absolute_Error of the XGB model on train set is 0.3%


In [162]:
#Keras

def buildTrain(train, past_hour, future_hour):
    """
    Create the dataset with time-windows
    """
    X_train, Y_train = [], []
    for i in range(train.shape[0] - future_hour - past_hour):
        X_train.append(np.array(train.iloc[i:i + past_hour]))
        Y_train.append(np.array(train.iloc[i + past_hour:i + past_hour + future_hour]["KWH"]))
    return np.array(X_train), np.array(Y_train)

In [233]:
def normalization(df):
    """
    Scaling the dataset
    """
    df = df.apply(lambda x :(x - np.min(x)) / (np.max(x) - np.min(x)))
    return df

In [385]:
# Many-To-Many Predict
# Predict 6 hours in the future using data from past 6 hours
lstm_features = ['KWH''1_days_before','3_days_before','7_days_before','month','day','hour']

print('Dataset Preparing...')
data_lstm = data_prep[lstm_features]
data_norm = normalization(data_lstm)
X_train,y_train = buildTrain(data_norm,past_hour = 6,future_hour = 6)
print('Dataset Preparing finished...')

In [386]:
X_train_lstm,X_test_lstm,y_train_lstm,y_test_lstm = train_test_split(X_train,
                                                y_train,
                                                test_size = 0.1,
                                                random_state = 8896)

In [387]:
def buildManyToManyModel(shape):
    """
    Create LSTM-RNN
    """
    model = Sequential()
    model.add(LSTM(16, input_shape = (shape[1], shape[2]),return_sequences=True))
    # output shape: (6, 1)
    model.add(TimeDistributed(Dense(1)))
    
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model

In [388]:
# Transform Target from 2 dimmension to 3 dimension
y_train_lstm = y_train_lstm[:,:,np.newaxis]
y_test_lstm = y_test_lstm[:,:,np.newaxis]


In [391]:
model = buildManyToManyModel(X_train_lstm.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train_lstm, y_train_lstm, epochs= 500, batch_size=128, validation_data=(X_test_lstm, y_test_lstm), callbacks=[callback])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_32 (LSTM)               (None, 6, 16)             1536      
_________________________________________________________________
time_distributed_15 (TimeDis (None, 6, 1)              17        
Total params: 1,553
Trainable params: 1,553
Non-trainable params: 0
_________________________________________________________________
Train on 7133 samples, validate on 793 samples
Epoch 1/500
7133/7133 [==============================] - 3s 424us/step - loss: 0.1072 - val_loss: 0.0628
Epoch 2/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0561 - val_loss: 0.0403
Epoch 3/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0384 - val_loss: 0.0302
Epoch 4/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0313 - val_loss: 0.0258
Epoch 5/500
7133/7133 [==============================] - 0s 30us/step - 

Epoch 73/500
7133/7133 [==============================] - 0s 32us/step - loss: 0.0184 - val_loss: 0.0169
Epoch 74/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0184 - val_loss: 0.0166
Epoch 75/500
7133/7133 [==============================] - 0s 33us/step - loss: 0.0182 - val_loss: 0.0163
Epoch 76/500
7133/7133 [==============================] - 0s 32us/step - loss: 0.0182 - val_loss: 0.0162
Epoch 77/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0182 - val_loss: 0.0163
Epoch 78/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0182 - val_loss: 0.0164
Epoch 79/500
7133/7133 [==============================] - 0s 32us/step - loss: 0.0181 - val_loss: 0.0162
Epoch 80/500
7133/7133 [==============================] - 0s 32us/step - loss: 0.0182 - val_loss: 0.0160
Epoch 81/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0182 - val_loss: 0.0161
Epoch 82/500
7133/7133 [==============================]

7133/7133 [==============================] - 0s 32us/step - loss: 0.0169 - val_loss: 0.0152
Epoch 151/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0168 - val_loss: 0.0155
Epoch 152/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0168 - val_loss: 0.0153
Epoch 153/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0168 - val_loss: 0.0151
Epoch 154/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0168 - val_loss: 0.0154
Epoch 155/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0168 - val_loss: 0.0151
Epoch 156/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0167 - val_loss: 0.0151
Epoch 157/500
7133/7133 [==============================] - 0s 33us/step - loss: 0.0168 - val_loss: 0.0162
Epoch 158/500
7133/7133 [==============================] - 0s 34us/step - loss: 0.0167 - val_loss: 0.0150
Epoch 159/500
7133/7133 [==============================] - 0

7133/7133 [==============================] - 0s 32us/step - loss: 0.0158 - val_loss: 0.0142
Epoch 228/500
7133/7133 [==============================] - 0s 32us/step - loss: 0.0157 - val_loss: 0.0142
Epoch 229/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0157 - val_loss: 0.0141
Epoch 230/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0157 - val_loss: 0.0141
Epoch 231/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0157 - val_loss: 0.0143
Epoch 232/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0157 - val_loss: 0.0142
Epoch 233/500
7133/7133 [==============================] - 0s 33us/step - loss: 0.0156 - val_loss: 0.0144
Epoch 234/500
7133/7133 [==============================] - 0s 33us/step - loss: 0.0156 - val_loss: 0.0146
Epoch 235/500
7133/7133 [==============================] - 0s 32us/step - loss: 0.0159 - val_loss: 0.0142
Epoch 236/500
7133/7133 [==============================] - 0

7133/7133 [==============================] - 0s 34us/step - loss: 0.0151 - val_loss: 0.0137
Epoch 305/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0151 - val_loss: 0.0136
Epoch 306/500
7133/7133 [==============================] - 0s 33us/step - loss: 0.0150 - val_loss: 0.0137
Epoch 307/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0151 - val_loss: 0.0147
Epoch 308/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0151 - val_loss: 0.0136
Epoch 309/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0150 - val_loss: 0.0144
Epoch 310/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0151 - val_loss: 0.0135
Epoch 311/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0151 - val_loss: 0.0135
Epoch 312/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0151 - val_loss: 0.0136
Epoch 313/500
7133/7133 [==============================] - 0

7133/7133 [==============================] - 0s 31us/step - loss: 0.0146 - val_loss: 0.0137
Epoch 382/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0146 - val_loss: 0.0136
Epoch 383/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0147 - val_loss: 0.0137
Epoch 384/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0146 - val_loss: 0.0133
Epoch 385/500
7133/7133 [==============================] - 0s 31us/step - loss: 0.0146 - val_loss: 0.0132
Epoch 386/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0146 - val_loss: 0.0133
Epoch 387/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0146 - val_loss: 0.0132
Epoch 388/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0146 - val_loss: 0.0133
Epoch 389/500
7133/7133 [==============================] - 0s 30us/step - loss: 0.0146 - val_loss: 0.0132
Epoch 390/500
7133/7133 [==============================] - 0

In [394]:
y_predict_test = model.predict(X_test_lstm)
y_predict_train = model.predict(X_train_lstm)
lstm_score_test = mape(y_predict_test,y_test_lstm)
lstm_score_train = mape(y_predict_train,y_train_lstm)


In [395]:
print(f'The Mean_Percentage_Absolute_Error of the LSTM model on validation set is {round(lstm_score_test,3) * 100}%')
print(f'The Mean_Percentage_Absolute_Error of the LSTM model on train set is {round(lstm_score_train,3) * 100}%')

The Mean_Percentage_Absolute_Error of the LSTM model on validation set is 8.3%
The Mean_Percentage_Absolute_Error of the LSTM model on train set is 8.5%
